In [3]:
%pip install -U iminizinc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install ortools

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [27]:
from ortools.linear_solver import pywraplp
import math

def main():
    # Data
    k=2
    n=8


    NN=[[-7, 6],
    [-2, -9],
    [2, 3],
    [-7, -6],
    [-7, 7],
    [-8, 6],
    [-2, 3],
    [-1, -9]
    ]

    # Solver
    # Create the mip solver with the SCIP backend.
    '''  solver = pywraplp.Solver.CreateSolver('SAT')
    if not solver:
        return
    '''
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        return

    # Variables
    #    array[1..k,1..2] of var int :c;
    #array[1..n,1..k] of var bool:y;
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    data = {}

    infinity = 10000#solver.infinity()
    
    for i in range(n):
        for j in range(k):
            data['y'] =[solver.IntVar(0, 1,'') ,solver.IntVar(0, 1,'') ]
            data['a'] =[solver.IntVar(0, infinity, '') , solver.IntVar(0, infinity, '')]
            data['b'] =[solver.IntVar(0, infinity, '') ,solver.IntVar(0, infinity, '') ]
    data["N"] = NN 

    for i in range(k):
        for j in range(2):
            data['c']= [solver.IntVar(-infinity, infinity, '') ,solver.IntVar(-infinity, infinity, '')]

    # Constraints
    for i in range(n):
        for j in range(k):
            
            solver.Add((data['N'][i,0]-data['c'][j,0])>=data['a'][i,j])
            solver.Add((data['N'][i,0]-data['c'][j,0])<=-1*data['a'][i,j])

            solver.Add((data['N'][i,1]-data['c'][j,1])>=data['c'][i,j])
            solver.Add((data['N'][i,1]-data['c'][j,1])<=-1*data['b'][i,j])


    for i in range(n):
        solver.Add(solver.Sum([data['y'][i, j] for j in range(k)]) == 1)



    # Each task is assigned to exactly one worker.
    for j in range(k):
        solver.Add(solver.Sum([data['y'][i, j] for i in range(n)]) >= 0)
        solver.Add(solver.Sum([data['y'][i, j] for i in range(n)]) <= n)

    # Objective
    objective = solver.Objective()
    for i in range(n):
        for j in range(k):
            objective.SetCoefficient(data['y'][i,j],(data['y'][i,j] + data['y'][i,j]) )

    objective.SetMinimization()
    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f'Total cost = {solver.Objective().Value()}\n')
        print(c)
        for i in range(n):
            for j in range(k):
            
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if y[i, j].solution_value() > 0.5:
                    print(f'Worker {i} assigned to task {j}.' +
                          f' Cost: {NN[i][j]}')
    else:
        print('No solution found.')


if __name__ == '__main__':
    main()

TypeError: list indices must be integers or slices, not tuple